In [1]:
# Imports
import os
import tensorflow as tf
import math
import matplotlib.pyplot as plt



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

***

# LambdaCallback

This callback is required when you need to call some custom function on any of the events, and the provided callbacks do not suffice. For instance, say you want to put your logs into a database.

```
tf.keras.callbacks.LambdaCallback(on_epoch_begin=None, 
                                  on_epoch_end=None, 
                                  on_batch_begin=None, 
                                  on_batch_end=None,    
                                  on_train_begin=None, 
                                  on_train_end=None, 
                                  **kwargs)
```

All the parameters of this callback expect a function which takes the arguments specified here:

- on_epoch_begin and on_epoch_end: epoch, logs
- on_batch_begin and on_batch_end: batch, logs
- on_train_begin and on_train_end: logs


This callback is called for all the events, and executes the custom functions based on the parameters passed.



***


## Preparing Dataset 

***

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## Define the Model

In [3]:
# Define a simple sequential model
def create_model():
  model = Sequential([
    Dense(512, activation='relu', input_shape=(784,), name = 'Layer-1'),
    Dropout(0.2),
    Dense(10, name = "Layer-2")
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Layer-1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
Layer-2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


***
## Lambda Callback




### Case 1: On_Epoch_Begin and On_Epoch_End

Let's say we want something to be printed (if you like you can do whatever you like) at the begining and at the end of an epoch.

In [4]:
from tensorflow.keras.callbacks import LambdaCallback

# This callback will cause a print at the begining of each epoch
epoch_begin_callback = LambdaCallback(
    on_epoch_begin = lambda epoch, logs: print('Starting Epoch {}!'.format(epoch+1))
)

# This callback will cause a print at the end of each epoch
epoch_end_callback = LambdaCallback(
    on_epoch_end = lambda epoch, logs: print('End of Epoch {}!\n\n'.format(epoch+1)) 
)

Let’s train the model with above callbacks.

In [5]:
# Train the model with the new callback
history = model.fit(train_images, 
          train_labels,  
          epochs=5,
          verbose = False,
          validation_data=(test_images, test_labels),
          callbacks = [epoch_begin_callback, epoch_end_callback])  # Pass callback to training

Starting Epoch 1!
End of Epoch 1!


Starting Epoch 2!
End of Epoch 2!


Starting Epoch 3!
End of Epoch 3!


Starting Epoch 4!
End of Epoch 4!


Starting Epoch 5!
End of Epoch 5!





### Case 2: On_Bpoch_Begin and On_Bpoch_End

Let's say we want something to be printed (if you like you can do whatever you like) at the begining and at the end of an epoch.


In [6]:
from tensorflow.keras.callbacks import LambdaCallback


# This callback will cause a print at the begining of each epoch
batch_begin_callback = LambdaCallback(
    on_batch_begin = lambda batch, logs:  print('Starting Batch {}!'.format(batch+1)) 
    )

# This callback will cause a print at the end of each epoch
batch_end_callback = LambdaCallback(
    on_batch_end = lambda batch, logs: print('The loss after batch {} is {:1f}!\n\n'.format(batch+1, logs['loss'])) 
)

Let’s train the model with above callbacks.

In [7]:
# Train the model with the new callback
history = model.fit(train_images, 
          train_labels,  
          epochs=2,
          verbose = False,
          validation_data=(test_images, test_labels),
          callbacks = [batch_begin_callback, batch_end_callback])  # Pass callback to training

Starting Batch 1!
The loss after batch 1 is 0.130452!


Starting Batch 2!
The loss after batch 2 is 0.100620!


Starting Batch 3!
The loss after batch 3 is 0.100316!


Starting Batch 4!
The loss after batch 4 is 0.101832!


Starting Batch 5!
The loss after batch 5 is 0.092769!


Starting Batch 6!
The loss after batch 6 is 0.089790!


Starting Batch 7!
The loss after batch 7 is 0.088184!


Starting Batch 8!
The loss after batch 8 is 0.086280!


Starting Batch 9!
The loss after batch 9 is 0.088275!


Starting Batch 10!
The loss after batch 10 is 0.089458!


Starting Batch 11!
The loss after batch 11 is 0.089218!


Starting Batch 12!
The loss after batch 12 is 0.092956!


Starting Batch 13!
The loss after batch 13 is 0.098612!


Starting Batch 14!
The loss after batch 14 is 0.112589!


Starting Batch 15!
The loss after batch 15 is 0.111510!


Starting Batch 16!
The loss after batch 16 is 0.108953!


Starting Batch 17!
The loss after batch 17 is 0.104702!


Starting Batch 18!
The loss afte

### Case 3: On_Train_Begin and On_Train_End



In [8]:
from tensorflow.keras.callbacks import LambdaCallback

train_begin_callback = LambdaCallback(
    on_train_begin=lambda logs: print('Training Started!\n...')
)

train_finish_callback = LambdaCallback(
    on_train_end=lambda logs: print('Training Finished! \nThe Training Loss is {:0.4f} \nThe Validation Loss is {:0.4f}'.
                                    format(logs['loss'], logs['val_loss']))
)

In [9]:
# Train the model with the new callback
history = model.fit(train_images, 
          train_labels,  
          epochs=2,
          verbose = False,
          validation_data=(test_images, test_labels),
          callbacks = [train_begin_callback, train_finish_callback])  # Pass callback to training

Training Started!
...
Training Finished! 
The Training Loss is 0.0517 
The Validation Loss is 0.4186


# References

1. [A Guide to TensorFlow Callbacks](https://blog.paperspace.com/tensorflow-callbacks/)
2. [Writing your own callbacks](https://www.tensorflow.org/guide/keras/custom_callback)
3. [tf.keras.callbacks.Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback)
4. [A Practical Introduction to Keras Callbacks in TensorFlow 2](https://towardsdatascience.com/a-practical-introduction-to-keras-callbacks-in-tensorflow-2-705d0c584966)
5. [How to use TensorFlow callbacks?](https://medium.com/ydata-ai/how-to-use-tensorflow-callbacks-f54f9bb6db25)
6. [Callbacks in Tensor Flow](https://sailajakarra.medium.com/callbacks-in-tensor-flow-f8e7f9996f5f)
7. [Keras Callbacks Explained In Three Minutes](https://www.kdnuggets.com/2019/08/keras-callbacks-explained-three-minutes.html)
8. [Learning Rate Schedule in Practice: an example with Keras and TensorFlow 2.0](https://towardsdatascience.com/learning-rate-schedule-in-practice-an-example-with-keras-and-tensorflow-2-0-2f48b2888a0c)
